# Image featurization ( part 3)

### Converting images to feature using transfer learning.
####  This script has used data11.csv file that is generated from script-2( part 2)
####   This script will product feature.npy, lst.npy file that will be used in part 4

In [0]:
#import all the necessary packages.

from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
import nltk
import math
import time
import re
import os

import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import requests
from PIL import Image
import pandas as pd
import pickle
import glob
from keras.preprocessing import image
#from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
from vgg16 import VGG16
from keras.preprocessing import image

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model

from datetime import datetime
from IPython.display import Image, display

In [0]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import keras
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
keras.backend.set_session(sess)

In [0]:
data11 = pd.read_csv('data11.csv')

In [0]:
data11.head(2)

,productId,title,imageUrlStr,mrp,categories,size,productBrand,path,exists
0,TOPE9ABBZU3HZRHN,Citrine Casual Short Sleeve Printed Women's Pi...,http://img.fkcdn.com/image/top/r/h/n/1-1-wwtpw...,1099,"Apparels>Women>Western Wear>Shirts, Tops & Tun...",S,Citrine,images/TOPE9ABBZU3HZRHN.jpeg,True
1,TOPE9ABBBTJYDSQE,Citrine Casual Short Sleeve Printed Women's Pi...,http://img.fkcdn.com/image/top/r/h/n/1-1-wwtpw...,1099,"Apparels>Women>Western Wear>Shirts, Tops & Tun...",M,Citrine,images/TOPE9ABBBTJYDSQE.jpeg,True


# I have used transfer learning for feature extraction from image. Used VGG net pretrained on ImageNet Data

In [0]:
image_input = Input(shape=(150, 150, 3))

model = VGG16(input_tensor=image_input, include_top= False,weights='imagenet')
model.layers[0].trainable = False
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

# Save the productId of all images in the lst( list) , that I have downloaded.

In [0]:
path = 'images/*'


In [0]:
lst = []
productId = pd.DataFrame()


for img_path in glob.glob(path):
    img = img_path.split('/')[1]
    img = img.split('.')[0]
    lst1 = []
    lst1.append(img)
    df1 = pd.DataFrame(lst1, columns=['productId'])
    productId = productId.append(df1)

cc = pd.DataFrame(productId.values)

### This If condition for constraining the number of images. Here I have put the condition infinity.

In [0]:
idxx = 0

for img_path in glob.glob(path):
    if idxx < 5000000:                  # This If condition for constraining the number of images. Here I have put the condition infinity.
        img = img_path.split('/')[1]
        img = img.split('.')[0]
        lst.append(img)
    idxx = idxx+1

In [0]:
idxx   # 346205 images.

346205

In [0]:
df_asins = list(data11['productId']) 

http://chrisschell.de/2018/02/01/how-to-efficiently-deal-with-huge-Numpy-arrays.html
### save the downloaed images productId in the lst.npy

In [0]:

np.save('lst.npy',lst)  

#### Initialize the numpy ndarray for saving the extracted features of 346205  images. Each image has extracted to 8192 length feature vector

In [0]:

resnet_feature_list = np.ones((346205,8192)) # prefilled array



In [0]:
resnet_feature_list

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [0]:
def extract_vector111(path):
    #resnet_feature_list = []
    idx = 0
    for img_path in glob.glob(path):
        if idx < 50000000:
            if idx% 1000 == 0:
                print(idx)
            img = image.load_img(img_path, target_size=(150, 150))
            img_data = image.img_to_array(img)
            img_data = np.expand_dims(img_data, axis=0)
            img_data = preprocess_input(img_data)
            resnet_feature = model.predict(img_data)
            resnet_feature_np = np.array(resnet_feature)
            resnet_feature_list[idx] = resnet_feature_np.flatten()
        
        idx = idx+1  
    return resnet_feature_list
            

In [0]:
start = datetime.now()
features = extract_vector111(path)
print("\nTime Taken: ",datetime.now() - start)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [0]:
np.save('features.npy',features)

### This script file has generated 2 file. one is list of productid of the downloaded images( lst.npy) and other is numpy array of images extracted features. (features.npy)